In [1]:
import json
import os
from random import randint
import shutil

In [2]:
#if new objects have been added, the META_JSON at bottom must be modified to account for it

coco_path = 'WUB814y5ns0_165_180_export_data.json'
#sv_path = 'Screenshot (509).png.json'
project_name = 'ground_truth_test'

In [3]:
# Load coco json
f = open(coco_path)
coco_file = json.load(f)

# Load supervisely json
#f2 = open(sv_path)
#sv_file = json.load(f2)


In [5]:
output_json = {}

for annot in coco_file['annotations']:

    #get image index
    for ind, image in enumerate(coco_file['images']):
        if image['id'] == annot['image_id']:
            image_index = ind
        
    #get image name
    image_name = coco_file['images'][image_index]['file_name']
    
    # create new key of filename in output dict if doesnt exist and do these one time operations
    if image_name not in output_json.keys():
        output_json[image_name] = {}
        output_json[image_name]['tags'] = []
        output_json[image_name]['size'] = {'height': coco_file['images'][image_index]['height'], 'width': coco_file['images'][image_index]['width']}
        output_json[image_name]['objects'] = []

    #annot dict to fill and later append to 'objects' - filled with static values on declaration
    temp_objects_dict = {'id': randint(10000000, 99999999), 'description': '', 
                                            'geometryType': 'polygon', 'labelerLogin': '', 
                                            'tags': [], 'points': {'exterior': [], 'interior': []}}

    #get class name and id
    temp_objects_dict['classId'] = annot['category_id']
    temp_objects_dict['classTitle'] = coco_file['categories'][annot['category_id'] - 1]['name']

    #coordinates
    coordinate_list = annot['segmentation']

    while len(coordinate_list) > 0:
        xy_list = [coordinate_list.pop(0), coordinate_list.pop(0)]
        temp_objects_dict['points']['exterior'].append(xy_list)

    output_json[image_name]['objects'].append(temp_objects_dict)


### For Ground Truth

In [4]:
output_json = {}

for annot in coco_file['annotations']:

    #get image index
    for ind, image in enumerate(coco_file['images']):
        if image['id'] == annot['image_id']:
            image_index = ind
        
    #get image name
    image_name = coco_file['images'][image_index]['file_name']
    
    # create new key of filename in output dict if doesnt exist and do these one time operations
    if image_name not in output_json.keys():
        output_json[image_name] = {}
        output_json[image_name]['tags'] = []
        output_json[image_name]['size'] = {'height': coco_file['images'][image_index]['height'], 'width': coco_file['images'][image_index]['width']}
        output_json[image_name]['objects'] = []

    #annot dict to fill and later append to 'objects' - filled with static values on declaration
    temp_objects_dict = {'id': randint(10000000, 99999999), 'description': '', 
                                            'geometryType': 'rectangle', 'labelerLogin': '', 
                                            'tags': [], 'points': {'exterior': [], 'interior': []}}

    #get class name and id
    temp_objects_dict['classId'] = annot['category_id']
    temp_objects_dict['classTitle'] = coco_file['categories'][annot['category_id']]['name']

    #coordinates
    coordinate_list = annot['bbox']

    coordinate_list[2] = coordinate_list[2] + coordinate_list[0]
    coordinate_list[3] = coordinate_list[3] + coordinate_list[1]

    xy_list = [int(coordinate_list[0]), int(coordinate_list[1])]
    xy_list2 = [int(coordinate_list[2]), int(coordinate_list[3])]
    
    temp_objects_dict['points']['exterior'].append(xy_list)
    temp_objects_dict['points']['exterior'].append(xy_list2)
 
   
    output_json[image_name]['objects'].append(temp_objects_dict)


In [5]:
#run this to nuke your pc and download all image jsons
os.makedirs(f'output/{project_name}/{project_name}', exist_ok=True)

for key in output_json.keys():
    with open(f'output/{project_name}/{project_name}/{key}.json', "w") as outfile:
        json.dump(output_json[key], outfile)

In [6]:
#creates meta json
meta_json = {
    "classes": [
        {
            'id': 1, 
            'title': 'uniformed',
            "shape": "polygon",
            "hotkey": "O",
            "color": "#FF69B4"
        },
        {
            'id': 2, 
            'title': 'non_uniformed',
            "shape": "polygon",
            'hotkey': 'C',
            "color": "#008000"
        },
        {
            "id": 3,
            "title": "riot_shield",
            "shape": "polygon",
            "hotkey": "R",
            "color": "#6FF6F1"
        },
        {
            'id': 4, 
            'title': 'gun',
            "shape": "polygon",
            'hotkey': 'G',
            'color': '#808080'
        },
        {
            'id': 5, 
            'title': 'pepper_spray',
            "shape": "polygon",
            'hotkey': 'S',
            "color": "#BF40BF"
        },
        {
            "id": 6,
            "title": "baton",
            "shape": "polygon",
            "hotkey": "B",
            "color": "#DB490F"
        },
        {
            'id': 7, 
            'title': 'chemical_smoke',
            "shape": "polygon",
            'hotkey': 'T',
            "color": "#FFBF00"
        }
    ],
    "tags": [
        {
            "id": 22919711,
            "name": "problem",
            "color": "#9F3868",
            "value_type": "none",
            "hotkey": "P",
            "applicable_type": "all",
            "classes": []
        },
    ],
    "projectType": "images"
}

In [7]:
#exports meta, zips it all, then removes output folder

with open(f'output/{project_name}/meta.json', "w") as outfile:
    json.dump(meta_json, outfile)

shutil.make_archive(project_name, 'zip', 'output')

shutil.rmtree('output')